In [12]:
import pandas as pd
full = pd.read_csv("../data/nba_games_rolling_averages.csv", index_col=0)

In [ ]:
# In train_rr_2.ipynb, right after loading
full = pd.read_csv("../data/nba_games_rolling_averages.csv")

print("Shape:", full.shape)
print("Total NaN values:", full.isnull().sum().sum())
print("NaN in target:", full["target"].isnull().sum())

Shape: (15774, 423)
Total NaN values: 0


KeyError: "['ts%_opp (last10)', 'fta_opp (last10)', 'trb_max_opp (last10)', 'trb% (last10)', '3p_opp (last10)', 'fta_max (last10)', 'ast (last10)', 'ftr_max (last10)', 'ft (last10)', 'trb%_max (last10)', 'trb%_max_opp (last10)', 'ft%_max_opp (last10)', '3p%_max_opp (last10)', 'drtg_max (last10)', 'fg%_max_opp (last10)', 'ortg (last10)', 'total (last10)', '3pa_max (last10)', 'pts_opp (last10)', 'tov%_opp (last10)', '3par_max (last10)', 'drb_max_opp (last10)', 'tov_opp (last10)', 'drb%_max (last10)', 'drtg (last10)', 'blk_max_opp (last10)', 'orb% (last10)', '3p_max (last10)', 'usg% (last10)', 'fta (last10)', '3par (last10)', 'stl%_max_opp (last10)', 'pts_max (last10)', 'ts%_max (last10)', 'mp_opp (last10)', 'tov (last10)', 'ast%_max_opp (last10)', 'drb%_opp (last10)', 'ast_max_opp (last10)', 'home (last10)', 'fga (last10)', 'stl_opp (last10)', 'usg%_opp (last10)', 'fg (last10)', 'fg%_opp (last10)', 'fga_opp (last10)', 'efg%_max_opp (last10)', 'ftr_max_opp (last10)', 'ft% (last10)', 'tov_

In [6]:
from sklearn.model_selection import TimeSeriesSplit     # import for feature selection (only want to train with certain data)
# ensures we are only using past data to predict future outcomes (not vice versa!)

from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import RidgeClassifier

rr = RidgeClassifier(alpha=0.1) # rr -> Ridge Regression (allows for classification) -> the MODEL itself
split = TimeSeriesSplit(n_splits=3) # cross-validation strategy, 3 splits for time series cross-validation

sfs = SequentialFeatureSelector(rr, n_features_to_select=30, direction="forward", cv=split) # feature selection strategy: will train the model (rr) using different sets of features

In [7]:
# logic to choose columns to use for training

removed_columns = list(full.columns[full.dtypes == "object"]) + ['season', 'date', 'won', 'target', 'team', 'team_opp']
selected_columns = full.columns[~full.columns.isin(removed_columns)] # negate columns in removed_columns

In [9]:
sfs.fit(full[selected_columns], full["target"]) # refit feature selector to new data with rolling averages

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
import joblib
joblib.dump(sfs, '../models/ridge_regression/feature_selector_optimized.pkl') # save the feature selector to a file for later use
joblib.dump(rr, '../models/ridge_regression/model_optimized.pkl') # save the feature selector to a file for later use